This notebook performs a series of thermal calculations to assess the loading capability of a transformer given various operating conditions.

This file is part of IEEE C57.91 2024 project which is released under BSD-3-Clause.
See file LICENSE.md or go to https://opensource.ieee.org/inslife/ieee-c57.91-2024/ for full license details.

In [ ]:
import transformer_thermal_models
from transformer_thermal_models import Transformer, LoadConditions

import pandas as pd

import numpy as np

In [ ]:
#Load transformer data
xfmr = Transformer()
xfmr.from_json('../Example_Data/transformer_thermal_nameplate.json')

#Load loading conditions file
lc = LoadConditions()


In [ ]:
initial_load = 0.5
max_T_hs = 110
permissable_estimated_loss_of_life = 0.05

condition_string = 'Initial Load = '+str(initial_load)+'; T_hs < '+str(max_T_hs)+' [C]'

test_load = np.flip(np.round(np.arange(initial_load, 2.1, 0.1), 3))
test_ambient = [-25,-20,-10,0,10,20,30,40]
test_time = [0.5,1.0,2.0,4.0,8.0,24.]

test_load,test_ambient,test_time

columns=pd.MultiIndex.from_product([[condition_string],test_ambient], names=['Maximum Permissible Overload:','Ambient Temperature [C]: '])
permissible_load_table = pd.DataFrame(columns=columns,index=pd.Index(test_time,name='Duration [Hours]'))

permissible_load_table

In [ ]:
for overload_time in test_time:
    last_overload_amount = np.inf
    for ambient in test_ambient:
        for overload_amount in test_load:
            print(overload_time,ambient,overload_amount)
                  
            if overload_amount-0.2 > last_overload_amount:
                # use the last permissible model to infer what 'hotter' step parameters can obvioulsy be skipped
                # can significantly reduce the computation time
                print("Skipping: ",overload_time,ambient,overload_amount)
                continue
                
            time = np.linspace(0,48*60,48*60)
            load = np.zeros(48*60)+initial_load
            T_ambient = np.zeros(48*60)+ambient
            
            load[24*60:24*60+int(60*overload_time)] = overload_amount

            lc.load = load
            lc.time = time
            lc.T_ambient = T_ambient
            #lc.overexcited = oe
            lc.update_profiles()
            
            transformer_thermal_models.solve_temperatures(xfmr,lc,'main_clause_7_diff')
            
            #criteria for "permissible overload"
            print(max(xfmr.solution['Hot Spot [C]']))
            if max(xfmr.solution['Hot Spot [C]']) < max_T_hs:
                permissible_load_table[condition_string][ambient][overload_time] = overload_amount
                print("Overload: ",overload_amount)
                last_overload_amount = overload_amount
                break
                
solved_permissible_load_table = permissible_load_table.copy()          

In [ ]:
solved_permissible_load_table.style.format(precision=1).format_index(precision=1)

In [ ]:
permissable_estimated_loss_of_life = 0.04
condition_string = 'Initial Load = '+str(initial_load)+'; T_hs < 180 [C]; %ELOL = '+str(permissable_estimated_loss_of_life*100)
columns=pd.MultiIndex.from_product([[condition_string],test_ambient], names=['Maximum Permissible Overload:','Ambient Temperature [C]: '])
permissible_load_table = pd.DataFrame(columns=columns,index=pd.Index(test_time,name='Duration [Hours]'))

In [ ]:
for overload_time in test_time:
    last_overload_amount = np.inf
    for ambient in test_ambient:
        for overload_amount in test_load:
            print(overload_time,ambient,overload_amount)
                  
            if overload_amount-0.2 > last_overload_amount:
                # use the last permissible model to infer what 'hotter' step parameters can obvioulsy be skipped
                # can significantly reduce the computation time
                print("Skipping: ",overload_time,ambient,overload_amount)
                continue
                
            time = np.linspace(0,48*60,48*60)
            load = np.zeros(48*60)+initial_load
            T_ambient = np.zeros(48*60)+ambient
            
            load[24*60:24*60+int(60*overload_time)] = overload_amount

            lc.load = load
            lc.time = time
            lc.T_ambient = T_ambient
            #lc.overexcited = oe
            lc.update_profiles()
            
            transformer_thermal_models.solve_temperatures(xfmr,lc,'main_clause_7_diff')
            
            xfmr = transformer_thermal_models.solve_estimated_loss_of_life(xfmr,lc)
            
            #criteria for "permissible overload"
            print(max(xfmr.solution['Est. Loss of Life']))
            if (max(xfmr.solution['Hot Spot [C]']) < 180) & (max(xfmr.solution['Est. Loss of Life']) < 4):
                permissible_load_table[condition_string][ambient][overload_time] = overload_amount
                print("Overload: ",overload_amount)
                last_overload_amount = overload_amount
                break
                
solved_permissible_load_table = permissible_load_table.copy()          

In [ ]:
solved_permissible_load_table.style.format(precision=1).format_index(precision=1)